In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from numpy import asarray

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_roc_curve, plot_confusion_matrix, confusion_matrix, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
!ls ../../data

features.csv target.csv


In [4]:
df = pd.read_csv('../../data/features.csv')

In [5]:
target = pd.read_csv('../../data/target.csv')

## Data Exploration

In [6]:
df = df.merge(target, on=['id'])

In [7]:
df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kiduruni,Kilimanjaro,3,5,Hai,Masama Magharibi,125,True,GeoData Consultants Ltd,Water Board,Losaa Kia water supply,True,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Igumbilo,Iringa,11,4,Njombe,Ikondo,56,True,GeoData Consultants Ltd,VWC,Ikondo electrical water sch,True,1996,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,Rufiji,Madungulu,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,Mwinyi,Dodoma,1,4,Chamwino,Mvumi Makulu,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


We have decided to drop installer because there are too many uniques and it could be just noise. We are also trying to prevent multicollinearity. 

#list(df['installer'].value_counts())

In [8]:
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [9]:
df.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

- Drop waterpoint_type and keep waterpoint_type_group. 
- Dropping scheme_name as well because there are so many nulls and there is not enough information. 
- drop funder 
- drop installer
- drop num_private there is no information given about it

In [10]:
df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [11]:
df['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

We dropped the following variables because there were too many different values and we wanted to get rid of any extra noise in our data frame.

In [12]:
final_df = df.drop(columns = ['id', 'waterpoint_type', 'scheme_name', 'funder',
        'num_private', 'permit', 'extraction_type', 'extraction_type_group',
        'region_code','district_code','lga','ward', 'recorded_by', 'public_meeting',
        'scheme_management', 'installer', 'payment', 'subvillage', 'wpt_name', 'date_recorded'])
final_df

,amount_tsh,gps_height,longitude,latitude,basin,region,population,construction_year,extraction_type_class,management,management_group,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type_group,status_group
0,6000.0,1390,34.938093,-9.856322,Lake Nyasa,Iringa,109,1999,gravity,vwc,user-group,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,functional
1,0.0,1399,34.698766,-2.147466,Lake Victoria,Mara,280,2010,gravity,wug,user-group,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional
2,25.0,686,37.460664,-3.821329,Pangani,Manyara,250,2009,gravity,vwc,user-group,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe,functional
3,0.0,263,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,58,1986,submersible,vwc,user-group,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe,non functional
4,0.0,0,31.130847,-1.825359,Lake Victoria,Kagera,0,0,gravity,other,other,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,1210,37.169807,-3.253847,Pangani,Kilimanjaro,125,1999,gravity,water board,user-group,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,functional
59396,4700.0,1212,35.249991,-9.070629,Rufiji,Iringa,56,1996,gravity,vwc,user-group,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,functional
59397,0.0,0,34.017087,-8.750434,Rufiji,Mbeya,0,0,handpump,vwc,user-group,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,functional
59398,0.0,0,35.861315,-6.378573,Rufiji,Dodoma,0,0,handpump,vwc,user-group,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,functional


In [13]:
final_df.isna().sum()

amount_tsh               0
gps_height               0
longitude                0
latitude                 0
basin                    0
region                   0
population               0
construction_year        0
extraction_type_class    0
management               0
management_group         0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type_group    0
status_group             0
dtype: int64

In [14]:
list(final_df['population'].values)

[109,
 280,
 250,
 58,
 0,
 1,
 0,
 0,
 0,
 0,
 345,
 250,
 0,
 1,
 0,
 200,
 35,
 50,
 1000,
 1,
 4,
 0,
 350,
 210,
 156,
 140,
 260,
 0,
 1,
 1,
 0,
 150,
 180,
 70,
 100,
 200,
 0,
 0,
 100,
 230,
 50,
 0,
 250,
 1,
 100,
 0,
 30,
 0,
 70,
 20,
 10,
 200,
 0,
 0,
 0,
 1,
 45,
 150,
 456,
 0,
 200,
 1,
 567,
 130,
 225,
 1,
 50,
 0,
 54,
 1,
 280,
 0,
 0,
 200,
 1,
 0,
 75,
 140,
 900,
 0,
 0,
 100,
 360,
 0,
 544,
 0,
 441,
 0,
 20,
 0,
 120,
 280,
 40,
 221,
 0,
 950,
 1430,
 0,
 0,
 0,
 110,
 256,
 150,
 1,
 75,
 320,
 0,
 0,
 1,
 0,
 120,
 120,
 0,
 0,
 0,
 0,
 90,
 804,
 600,
 1,
 0,
 0,
 0,
 370,
 0,
 0,
 50,
 305,
 0,
 0,
 0,
 150,
 0,
 0,
 1,
 50,
 0,
 1,
 0,
 0,
 0,
 150,
 0,
 1,
 1,
 150,
 0,
 40,
 200,
 0,
 1600,
 0,
 50,
 400,
 590,
 1,
 0,
 0,
 0,
 1,
 250,
 450,
 80,
 560,
 0,
 2500,
 0,
 1050,
 0,
 36,
 540,
 300,
 12,
 450,
 632,
 50,
 245,
 0,
 0,
 600,
 1,
 32,
 120,
 59,
 230,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 50,
 25,
 150,
 500,
 1,
 100,
 0,
 0,
 1,
 40,
 40,
 0

In [15]:
near_pop = []
for x in list(final_df['population'].values): 
    if x == 0: 
        near_pop.append(False)
    else: 
        near_pop.append(True)

In [16]:
final_df['near_pop'] = near_pop

We created a new column, near_pop, so that we could see if there was a correlation between the functionality of the water well and the population around it. 

In [17]:
type(final_df)

pandas.core.frame.DataFrame

In [18]:
final_df.head()

,amount_tsh,gps_height,longitude,latitude,basin,region,population,construction_year,extraction_type_class,management,management_group,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type_group,status_group,near_pop
0,6000.0,1390,34.938093,-9.856322,Lake Nyasa,Iringa,109,1999,gravity,vwc,user-group,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,functional,True
1,0.0,1399,34.698766,-2.147466,Lake Victoria,Mara,280,2010,gravity,wug,user-group,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional,True
2,25.0,686,37.460664,-3.821329,Pangani,Manyara,250,2009,gravity,vwc,user-group,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe,functional,True
3,0.0,263,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,58,1986,submersible,vwc,user-group,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe,non functional,True
4,0.0,0,31.130847,-1.825359,Lake Victoria,Kagera,0,0,gravity,other,other,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional,False


In [19]:
final_df['near_pop'].value_counts()

True     38019
False    21381
Name: near_pop, dtype: int64

## Business Problem

- wells that need work will be labeled nonfunctional 
- will air on the side of caution and look to prevent false positives (says the water well is working when it is not)

## Model 1: KNN

In [20]:
X = final_df.drop(columns = ['status_group'])
y = final_df['status_group']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

In [21]:
#https://stackoverflow.com/questions/61641852/what-is-the-valid-specification-of-the-columns-needed-for-sklearn-classifier-p

cat_cols = X.select_dtypes(include="object").columns
num_cols = X.select_dtypes(exclude="object").columns

In [22]:
num_trans = Pipeline(steps=[
    ('scaler', StandardScaler())
])
cat_trans = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first',
                          sparse=False))
])

In [23]:
pp = ColumnTransformer(transformers=[
    ('num', num_trans, num_cols),
    ('cat', cat_trans, cat_cols)
])

In [24]:
pp.fit(X_train)
X_tr_pp = pp.transform(X_train)
X_te_pp = pp.transform(X_test)

In [25]:
knn = KNeighborsClassifier(3)

knn.fit(X_tr_pp, y_train)

KNeighborsClassifier(n_neighbors=3)

In [26]:
scores = cross_val_score(estimator=knn, X=X_tr_pp,
                y=y_train, cv=5)
print(f"Median score: {np.median(scores):.4f} (+/- {np.std(scores):.4f})")

Median score: 0.7617 (+/- 0.0060)


In [27]:
knn.score(X_te_pp, y_test)

0.7584511784511785

## Model 2: Decision Tree

In [28]:
dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_tr_pp, y_train)

DecisionTreeClassifier(random_state=42)

In [29]:
scores = cross_val_score(estimator=dt, X=X_tr_pp,
                         y=y_train, cv=5)
print(f"Median score: {np.median(scores):.4f} (+/- {np.std(scores):.4f})")

Median score: 0.7448 (+/- 0.0026)


In [30]:
dt.score(X_te_pp, y_test)

0.7501683501683502

## Model 3: Logistic Regression

In [31]:
lr = LogisticRegression(max_iter=1000, random_state=42)

lr.fit(X_tr_pp, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [32]:
scores = cross_val_score(estimator=lr, X=X_tr_pp,
                         y=y_train, cv=5)
print(f"Median score: {np.median(scores):.4f} (+/- {np.std(scores):.4f})")

Median score: 0.7256 (+/- 0.0042)


In [33]:
lr.score(X_te_pp, y_test)

0.7245117845117846

## Model 4: Random Forest 

In [56]:
rfc = RandomForestClassifier(random_state=42, max_depth=22, min_samples_leaf=14)

rfc.fit(X_tr_pp, y_train)

RandomForestClassifier(max_depth=22, min_samples_leaf=14, random_state=42)

In [57]:
scores = cross_val_score(estimator=rfc, X=X_tr_pp,
                         y=y_train, cv=9)
print(f"Median score: {np.median(scores):.4f} (+/- {np.std(scores):.4f})")

Median score: 0.7699 (+/- 0.0067)


In [58]:
rfc.score(X_te_pp, y_test)

0.7655892255892256

## Model 5: XG Boost 

In [37]:
xgb_model = xgb.XGBClassifier(random_state=42, learning_rate=1, max_depth=6)

xgb_model.fit(X_tr_pp, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [38]:
print(f"Train Score: {xgb_model.score(X_tr_pp, y_train)}")
print(f"Test Score: {xgb_model.score(X_te_pp, y_test)}")

Train Score: 0.9114927048260382
Test Score: 0.791986531986532


## Model 6: Naive Bayes (Bernoulli)

In [39]:

nb = BernoulliNB()

nb.fit(X_tr_pp,y_train)

train_preds = nb.predict(X_tr_pp)
y_preds = nb.predict(X_te_pp)


print(f'Naive Bayes Train Accuracy: {accuracy_score(y_train, train_preds):.4f}')
print(f'Naive Bayes Test Accuracy: {accuracy_score(y_test, y_preds):.4f}')


Naive Bayes Train Accuracy: 0.6133
Naive Bayes Test Accuracy: 0.6105


## Tuning with GridSearchCV

### Random Forest Model

In [50]:
rfc_cv = Pipeline(steps=[
    ('preprocessor', pp), 
    ('tree', DecisionTreeClassifier(max_depth=7, random_state=713))
])

In [51]:
param_grid = {
    'tree__max_depth':[5, 14, 22, 10], 
    'tree__min_samples_leaf':[5, 14, 22]
}

In [54]:
gs_rfc = GridSearchCV(rfc_cv, param_grid, cv=5, scoring='accuracy')


gs_rfc.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population',
       'construction_year', 'near_pop'],
      dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('ohe',
                                                                                          OneHotEncoder(drop='first',
                                                                                                        sparse=False))]),
     

In [55]:
gs_rfc.best_params_

{'tree__max_depth': 22, 'tree__min_samples_leaf': 14}

### Naive Bayes (Bernoulli)

In [103]:
nb_cv = Pipeline(steps=[
    ('preprocessor', pp), 
    ('NB', BernoulliNB())
])

In [104]:
param_nb = {
    'NB__alpha':[0, 0.5, 1], 
    'NB__fit_prior':[True, False]
}

In [105]:
gs_nb = GridSearchCV(nb_cv, param_nb, cv=5, scoring='accuracy')


gs_nb.fit(X_train, y_train)

/Users/reed/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/reed/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/reed/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/reed/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/reed/

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population',
       'construction_year', 'near_pop'],
      dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('ohe',
                                                                                          OneHotEncoder(drop='first',
                                                                                                        sparse=False))]),
     

In [106]:
gs_nb.best_params_

{'NB__alpha': 0.5, 'NB__fit_prior': True}